In [1]:
!git clone https://github.com/CarperAI/trlx.git
!git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

Cloning into 'trlx'...
remote: Enumerating objects: 8089, done.
remote: Counting objects: 100% (3366/3366), done.
remote: Compressing objects: 100% (764/764), done.
remote: Total 8089 (delta 2857), reused 2896 (delta 2600), pack-reused 4723
Receiving objects: 100% (8089/8089), 46.84 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (5563/5563), done.
Obtaining file:///content/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.3/867.3 kB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [2]:
# uninstall scikit_learn + jax to avoid numpy issues
!pip uninstall -y scikit_learn jax

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: jax 0.4.14
Uninstalling jax-0.4.14:
  Successfully uninstalled jax-0.4.14


In [3]:
import os

# run within repo
os.chdir('/content/trlx')
print(os.getcwd())

/content/trlx


In [4]:
import yaml
from datasets import load_dataset
from transformers import pipeline
import pathlib
from typing import Dict, List
import trlx
from trlx.data.default_configs import TRLConfig, default_ilql_config

[2023-09-27 17:13:07,310] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
default_config = default_ilql_config().to_dict()
default_config['train']['tracker'] = None
default_config['train']['batch_size'] = 16
default_config['train']['epochs'] = 10
config = TRLConfig.update(default_config, {})
print(config)

{
    "method": {
        "name": "ilqlconfig",
        "tau": 0.7,
        "gamma": 0.99,
        "cql_scale": 0.1,
        "awac_scale": 1,
        "alpha": 0.001,
        "beta": 0,
        "steps_for_target_q_sync": 5,
        "two_qs": true,
        "gen_kwargs": {
            "max_new_tokens": 56,
            "top_k": 20,
            "beta": 1,
            "temperature": 1.0
        }
    },
    "model": {
        "model_path": "gpt2",
        "model_arch_type": "causal",
        "num_layers_unfrozen": -1,
        "peft_config": null
    },
    "optimizer": {
        "name": "adamw",
        "kwargs": {
            "lr": 5e-05,
            "betas": [
                0.9,
                0.95
            ],
            "eps": 1e-08,
            "weight_decay": 1e-06
        }
    },
    "scheduler": {
        "name": "cosine_annealing",
        "kwargs": {
            "T_max": 1000000000000.0,
            "eta_min": 5e-05
        }
    },
    "tokenizer": {
        "tokenizer_path

In [7]:
def get_positive_score(scores):
    "Extract value associated with a positive sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]

sentiment_fn = pipeline(
    "sentiment-analysis",
    "lvwerra/distilbert-imdb",
    top_k=2,
    truncation=True,
    batch_size=256,
    device=0,
)

def metric_fn(samples: List[str], **kwargs) -> Dict[str, List[float]]:
    sentiments = list(map(get_positive_score, sentiment_fn(samples)))
    return {"sentiments": sentiments}

imdb = load_dataset("imdb", split="train+test")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
trainer = trlx.train(
    samples=imdb["text"],
    rewards=imdb["label"],
    eval_prompts=[
        "I don't know much about Hungarian underground",
        "What made this movie so distinctly",
        "Like the sandwich I just bought at the grocery store,",
        "I cannot believe how much this movie made me want to"
    ] * 20,
    metric_fn=metric_fn,
    config=config,
)

[RANK 0] Initializing model: gpt2


Using pad_token, but it is not set yet.
[RANK 0] Collecting rollouts
Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 1024). Running this sequence through the model will result in indexing errors
[RANK 0] Logging sample example


                                                  Sample Example                                                   
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Prompt        ┃ Response                                                                               ┃ Reward ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ <|endoftext|> │ I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that   │ 0      │
│               │ surrounded it when it was first released in 1967. I also heard that at first it was    │        │
│               │ seized by U.S. customs if it ever tried to enter this country, therefore being a fan   │        │
│               │ of films considered "                                                                  │        │
└───────────────┴────────────────────────────────────────────────────────────────────────────────────────┴────────┘

[RANK 0] Logging experience string statistics


     Experience String Stats (mean ∈ [min, max])      
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt Length ┃ Output Length   ┃ Sample Length    ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 1.00 ∈ [1, 1] │ 62.76 ∈ [9, 63] │ 63.76 ∈ [10, 64] │
└───────────────┴─────────────────┴──────────────────┘

...

                                      Evaluation #9 metrics/sentiments: 0.558                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                                        ┃ output                                             ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ I don't know much about Hungarian underground │  cinema, but I have seen it many times and have    │ 0.98       │
│                                               │ never missed something. I love the original        │            │
│                                               │ Hungarian original Hungarian classic "Viktor". I   │            │
│                                               │ have seen a few times on Hungarian video channels  │            │
│                                               │ (Viktor on Sotokon, Viktor on Kölsk                │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ What made this movie so distinctly            │  British? I thought this was a great British movie │ 0.97       │
│                                               │ about a British woman whose parents died during    │            │
│                                               │ the Civil War (with some British accent, as we'll  │            │
│                                               │ see in this documentary). It's great to see        │            │
│                                               │ British actors as part of such films, even though  │            │
│                                               │ most of the people who made                        │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ Like the sandwich I just bought at the        │  store, this was a wonderful movie. I think the    │ 0.559      │
│                                               │ only reason I gave it 4 stars is because of the    │            │
│                                               │ way the movie ended, they made this look bad and I │            │
│                                               │ think the reason they didn't make it 4 stars was   │            │
│                                               │ probably because the actors were all terrible and  │            │
│                                               │ the plot wasn                                      │            │
└───────────────────────────────────────────────┴────────────────────────────────────────────────────┴────────────┘

[RANK 0] Saving intermediate optimizer & model checkpoint into ckpts/checkpoint_1000
[RANK 0] Saving pretrained model into ckpts/checkpoint_1000/hf_model
[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/5]:   0%|          | 0/5 [00:00<?, ?it/s]

[RANK 0] Computing metrics
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
[RANK 0] Summarizing evaluation


                                      Evaluation #10 metrics/sentiments: 0.52                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                                        ┃ output                                             ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ I don't know much about Hungarian underground │  cinema but when i saw the film I was very excited │ 0.986      │
│                                               │ to try it!<br /><br />I rented the entire movie    │            │
│                                               │ and the whole movie was very interesting. The plot │            │
│                                               │ is simple and clever. <br /><br />I can't believe  │            │
│                                               │ i was so excited for                               │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ What made this movie so distinctly            │  enjoyable was the fact that the director didn't   │ 0.927      │
│                                               │ put his name on the cover, he was just going for a │            │
│                                               │ laugh. I saw that on the cover of my DVD player, I │            │
│                                               │ can't believe they even tried to hide his voice (I │            │
│                                               │ didn't think he was playing "The                   │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ Like the sandwich I just bought at the        │  end of every night, the "pizza" at the store      │ 0.00712    │
│                                               │ looks like a cheap version of a cheeseburger -     │            │
│                                               │ it's a piece of crap with no taste, no chemistry,  │            │
│                                               │ and the same kind of cheese as you have in your    │            │
│                                               │ home for a good cheeseburger.                      │            │
└───────────────────────────────────────────────┴────────────────────────────────────────────────────┴────────────┘

In [9]:

# output
input_str = 'One thing you should know about When Sally Met Harry is that'
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output))

One thing you should know about When Sally Met Harry is that this is one of the funniest shows out there. So, in my opinion, it should be shown on Troma. Not that Troma has much of a reputation. But I do like it when they try to keep it up with the most annoying show
